**This notebook is to test our model on live or recorded speech.**

**function specch_to_text(audiopath,record) takes either audio path or record flag (to record audio with duration 5 seconds)**

**Just run the notebook and test with specch_to_text(audiopath,record) function to return prediction**

In [10]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [2]:
%%capture
!pip install datasets==2.1.0
!pip install transformers==4.18.0
!pip install torchaudio
!pip install librosa
!pip install jiwer
!pip install ar-corrector

In [3]:
from IPython.display import Javascript
from google.colab import output
from base64 import b64decode
from io import BytesIO
!pip -q install pydub
from pydub import AudioSegment

RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""
def record(sec=3):
  display(Javascript(RECORD))
  s = output.eval_js('record(%d)' % (sec*1000))
  b = b64decode(s.split(',')[1])
  audio = AudioSegment.from_file(BytesIO(b))
  return audio

In [46]:
import pandas as pd
from datasets import Dataset


In [36]:
import torch
import torchaudio
from datasets import load_dataset
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
'''
get the model from this drive link https://drive.google.com/drive/folders/1-LKd7eWYIHC0XNQTZoer2o_3kpamZgyB?usp=share_link
and add pass the path to the model below

get the processor from the this drive link https://drive.google.com/drive/folders/11HhaXk0_bBHz_XI6KdTZ8U1vJhQp2rJz?usp=share_link
and add pass the path to the processor below
'''



model = Wav2Vec2ForCTC.from_pretrained("/content/drive/MyDrive/Final_speech/22new_wav2vec2-large-xlsr-egyptian-demo/checkpoint-17700") 
processor = Wav2Vec2Processor.from_pretrained("/content/drive/MyDrive/Final_speech/final_processor")



def speech_file_to_array_fn(batch):
    
    speech_array, sampling_rate = torchaudio.load(batch["audio"])
    resampler = torchaudio.transforms.Resample(sampling_rate, 16_000) 
    batch["audio"] = resampler(speech_array).squeeze().numpy()
    return batch




Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [48]:
def specch_to_text(audiofile='',record=0):
  if audiofile!='':
    path=audiofile
  elif record:
       print("recording start for 5 second,speak please")
       audio=record(sec=5)
       audio.export(out_f = "output.wav",
                        format = "wav")
       path="output.wav"
  else :
      "no audio file detected "
      return
  dftest = pd.DataFrame(columns=['audio'])
  dftest['audio']=[path]  
  test_dataset1 = Dataset.from_pandas(dftest)

  test_dataset = test_dataset1.map(speech_file_to_array_fn)
  inputs = processor(test_dataset["audio"], sampling_rate=16_000, return_tensors="pt", padding=True)

  with torch.no_grad():
      logits = model(inputs.input_values, attention_mask=inputs.attention_mask).logits

  predicted_ids = torch.argmax(logits, dim=-1)
  pred=processor.batch_decode(predicted_ids)
  pred= pred[0].replace('ى','ي')
  return pred
  


In [56]:
print(specch_to_text(audiofile='/content/2260.wav',record=0))

  0%|          | 0/1 [00:00<?, ?ex/s]

السنه الي فات سنه الفين وعشرين شهر اربعه خمسه كدا يا دوبك بس كنا بنروح كم يوم ونرجع
